# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherdf= "weatherdf.csv"
weatherdf = pd.read_csv(weatherdf)

weatherdf

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City country,City datetime
0,kaitangata,-46.28,169.85,54.05,77,64,6.73,NZ,1584507986
1,beringovskiy,63.05,179.32,28.85,90,99,6.35,RU,1584508034
2,kuldiga,56.97,21.96,42.80,93,90,17.22,LV,1584508034
3,atuona,-9.80,-139.03,82.49,78,99,12.80,PF,1584507998
4,jamestown,42.10,-79.24,37.99,95,75,6.93,US,1584508034
...,...,...,...,...,...,...,...,...,...
568,hammerfest,70.66,23.68,26.60,92,75,14.99,NO,1584508104
569,betulia,6.11,-75.98,60.80,93,75,2.24,CO,1584508104
570,fort payne,34.44,-85.72,60.01,100,1,5.57,US,1584508068
571,magway,20.15,94.92,95.99,18,0,8.46,MM,1584508105


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["City Latitude", "City Longitude"]].astype(float)

#Humidity as weights
humidity = weatherdf["Humidity"].astype(float)
locations

maxintensity= weatherdf['Humidity']
maxvalue= max(maxintensity)
maxvalue


100

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=maxvalue,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#flitered weather conditions for temperature between 70 and 80 degrees, wind speed <10, and no cloudiness
filtereddf=weatherdf[(weatherdf['Max Temperature']<80) & (weatherdf['Max Temperature']>70) & (weatherdf['Wind Speed']<10) & (weatherdf['Cloud coverage']==0)]

filtereddf.head(20)

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City country,City datetime
11,petatlan,17.52,-101.27,79.07,75,0,7.43,MX,1584508035
36,cienfuegos,22.15,-80.44,73.40,83,0,9.17,CU,1584508038
38,san patricio,19.22,-104.70,74.93,81,0,4.09,MX,1584508038
99,les cayes,18.20,-73.75,77.63,64,0,6.93,HT,1584508046
146,ormara,25.21,64.64,76.73,60,0,1.99,PK,1584508052
228,binzhou,37.37,118.02,78.01,18,0,5.99,CN,1584508062
344,ginir,7.13,40.70,70.25,54,0,6.04,ET,1584508076
383,sur,22.57,59.53,78.89,30,0,5.37,OM,1584508081
393,gwadar,25.12,62.33,75.47,61,0,1.28,PK,1584508082
460,west bay,19.37,-81.42,79.00,80,0,1.99,KY,1584508091


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store previous filtered DataFrame into variable named hotel_df
hotel_df=filtereddf

#Add a 'Hotel Name' column to the DataFrame
hotel_df['Hotel Name'] = ""
                                


# geocoordinates

target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["City Latitude"]
    lng = row["City Longitude"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except:
        #drop rows that return no hotel names
         hotel_df.loc[index, "Hotel Name"] = "null"
         hotel_df.drop(hotel_df.loc[hotel_df['Hotel Name']=='null'].index, inplace=True)
#max ten rows for markers
kl= hotel_df.iloc[0:10]        
kl.head(20)


            
        
    




C:\Users\Nabil\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Nabil\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\Nabil\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/inde

,City,City Latitude,City Longitude,Max Temperature,Humidity,Cloud coverage,Wind Speed,City country,City datetime,Hotel Name
11,petatlan,17.52,-101.27,79.07,75,0,7.43,MX,1584508035,Hotel Del Angel
36,cienfuegos,22.15,-80.44,73.40,83,0,9.17,CU,1584508038,Hotel La Unión
38,san patricio,19.22,-104.70,74.93,81,0,4.09,MX,1584508038,Grand Isla Navidad Resort
99,les cayes,18.20,-73.75,77.63,64,0,6.93,HT,1584508046,Hôtel Le Méridien
146,ormara,25.21,64.64,76.73,60,0,1.99,PK,1584508052,Zero Point Ormara
228,binzhou,37.37,118.02,78.01,18,0,5.99,CN,1584508062,JinJiang Inn Binzhou Huanhesan Road
344,ginir,7.13,40.70,70.25,54,0,6.04,ET,1584508076,Ijole Bale Hotel
383,sur,22.57,59.53,78.89,30,0,5.37,OM,1584508081,Sur Plaza Hotel
393,gwadar,25.12,62.33,75.47,61,0,1.28,PK,1584508082,Zaver Pearl Continental Hotel
460,west bay,19.37,-81.42,79.00,80,0,1.99,KY,1584508091,"The Ritz-Carlton, Grand Cayman"


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{City country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in kl.iterrows()]
locations = kl[["City Latitude", "City Longitude"]]

In [8]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_locations = locations
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))